In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-20"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
4306,2024-02-20,Argentina Liga Feminina,18:00,Ferro F,Quimsa F,1.12,5.56,126.5,1.85,1.85,-13.5,1.95,1.72,zZKxskc3,0.892857,0.179856,0.540541,0.540541,0.072713,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,81.42,0.00,0.0,0.0,0.0,0.0,0.939986,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
4307,2024-02-20,Argentina Liga Feminina,18:00,Obras F,Ciclista Olimpico La Banda F,1.09,23.00,125.5,1.85,1.85,-24.5,1.95,1.72,drMYsVsc,0.917431,0.043478,0.540541,0.540541,-0.039091,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,101.64,800.00,0.0,1.0,0.0,0.0,1.286236,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
4308,2024-02-20,Bahrein Premier League,13:30,Sitra Club,ISSA Town,9.35,1.04,159.5,1.81,1.85,6.5,1.82,1.82,WbPSw31I,0.106952,0.961538,0.552486,0.540541,0.068490,0.0,0.0,NaN,NaN,633.482,691.362644,1.091369,341.130,290.969877,0.852959,0.000,0.000,186.88,127.89,0.0,0.0,0.0,0.0,1.131099,0.015456,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
4309,2024-02-20,China Wcba Feminina,08:30,Hebei F,Daqing F,1.09,6.57,148.5,1.86,1.84,-20.5,1.95,1.74,CMpmcCLt,0.917431,0.152207,0.537634,0.543478,0.069638,0.4,0.0,NaN,NaN,407.022,428.746706,1.053375,250.726,194.138867,0.774307,463.398,0.000,1136.00,98.34,1.0,0.0,0.0,0.0,1.011735,0.007644,0.080484,2.65,0.530,0.169811,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
4310,2024-02-20,China Wcba Feminina,08:30,Shanxi Xing Rui F,Jiangsu Phoenix F,1.82,1.90,148.5,1.83,1.87,-2.5,2.06,1.65,UcuhdWzm,0.549451,0.526316,0.546448,0.534759,0.075766,0.4,0.6,NaN,NaN,126.968,53.195085,0.418965,541.766,824.557212,1.521980,120.734,540.874,88.88,1980.00,0.0,1.0,0.0,0.0,0.030413,0.015289,0.156288,-1.71,-0.342,-2.397661,0.000000,0.0,0.000000,16.01,3.202,0.281074,0.000000,0.0,0.000000
4311,2024-02-20,China Wcba Feminina,08:30,Tianjin F,Henan F,1.08,6.74,154.5,1.83,1.87,-17.5,1.98,1.74,GWGoyupj,0.925926,0.148368,0.546448,0.534759,0.074294,0.0,0.0,NaN,NaN,301.874,147.631902,0.489051,453.172,235.041999,0.518660,0.000,0.000,259.72,303.84,0.0,0.0,0.0,0.0,1.023587,0.015289,0.091240,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
4312,2024-02-20,China Wcba Feminina,08:30,Xiamen F,Guangdong F,11.60,21.00,149.5,1.85,1.85,33.5,1.98,1.72,ADbL1hDP,0.086207,0.047619,0.540541,0.540541,-0.866174,0.0,0.4,NaN,NaN,501.364,298.059980,0.594498,133.300,93.385028,0.700563,399.076,130.522,423.30,101.91,1.0,0.0,0.0,0.0,0.407779,0.000000,0.099377,-2.32,-0.464,-22.844828,0.000000,0.0,0.000000,-1.65,-0.330,-60.606061,0.000000,0.0,0.000000
4313,2024-02-20,Croácia Premijer Liga,14:00,GKK Sibenka,Dubrava,1.34,3.08,160.5,1.84,1.86,-7.5,1.98,1.74,MeGwrdNe,0.746269,0.324675,0.543478,0.537634,0.070944,1.0,0.8,NaN,NaN,178.098,90.612194,0.508777,205.062,33.734116,0.164507,205.212,316.954,330.26,170.94,1.0,0.0,0.0,0.0,0.556727,0.007644,0.091240,0.79,0.158,2.151899,0.000000,0.0,0.000000,-2.32,-0.464,-4.482759,0.000000,0.0,0.000000
4314,2024-02-20,Eua Ncaa,20:30,Villanova,Butler,1.37,3.04,139.5,1.83,1.89,-7.5,1.98,1.75,zBn3TBPL,0.729927,0.328947,0.546448,0.529101,0.058874,0.6,0.6,NaN,NaN,123.682,25.768627,0.208346,279.542,197.185736,0.705389,118.418,332.834,124.00,542.50,0.0,0.0,0.0,0.0,0.535541,0.022810,0.087204,-1.06,-0.212,-1.745283,0.715907,0.6,-0.115907,5.89,1.178,1.731749,0.000000,0.0,0.000000
4315,2024-02-20,Eua Ncaa,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
